In [115]:
import pandas as pd
import numpy as np
data=pd.read_csv(r'E:\Tai Lieu Hoc Phan\Nam3_KH2\Khoa Học Dữ Liệu\titanic.csv')


Xử lý dữ liệu trống

In [116]:
data=data[['age','fare','survived']]

# Mean, Median , Mode
median=data['age'].median()
data['age_median']=data['age'].fillna(median)

mean=round(data['age'].mean(),2)
data['age_mean']=data['age'].fillna(mean)

mode=data['age'].mode()[0]
data['age_mode']=data['age'].fillna(mode)

# Random sample imputation
random_samples = data['age'].dropna().sample(n=data['age'].isnull().sum(),random_state=0)
random_samples.index=data[data['age'].isnull()].index
data["age_random"]=data["age"]
data.loc[data['age'].isnull(),'age_random']=random_samples

# End of distribution imputation
extreme=round(mean+3*data['age'].std(),2)
data['age_end_dist']=data['age'].fillna(extreme)

# Arbitrary Value Imputation
random = np.random.randint(0,80,size=data['age'].isnull().sum())

data["age_arbitrary"]=data["age"]
data.loc[data['age'].isnull(),'age_arbitrary']=random
# Create a new feature
data["age_NaN"]=data["age"]
data['age_NaN']=data['age'].fillna(1)
data.loc[data['age']>0,'age_NaN'] = 0

data.head(10)


,age,fare,survived,age_median,age_mean,age_mode,age_random,age_end_dist,age_arbitrary,age_NaN
0,22.0,7.2500,0,22.0,22.0,22.0,22.0,22.00,22.0,0.0
1,38.0,71.2833,1,38.0,38.0,38.0,38.0,38.00,38.0,0.0
2,26.0,7.9250,1,26.0,26.0,26.0,26.0,26.00,26.0,0.0
3,35.0,53.1000,1,35.0,35.0,35.0,35.0,35.00,35.0,0.0
4,35.0,8.0500,0,35.0,35.0,35.0,35.0,35.00,35.0,0.0
5,NaN,8.4583,0,28.0,29.7,24.0,28.0,73.28,33.0,1.0
6,54.0,51.8625,0,54.0,54.0,54.0,54.0,54.00,54.0,0.0
7,2.0,21.0750,0,2.0,2.0,2.0,2.0,2.00,2.0,0.0
8,27.0,11.1333,1,27.0,27.0,27.0,27.0,27.00,27.0,0.0
9,14.0,30.0708,1,14.0,14.0,14.0,14.0,14.00,14.0,0.0


Xử lý ngoại lệ


In [117]:

data_pre=data.copy()
# Xử lý ngoại lệ fare
excep=data['fare'].sort_values().values
P25=excep[round(0.25*len(excep))]
P75=excep[round(0.75*len(excep))]
IQR=P75-P25
upper=P75+3*IQR
lower=P25-3*IQR
data.loc[data['fare']>upper,'fare']=upper
data.loc[data['fare']<lower,'fare']=lower

# Xử lý ngoại lệ age 
str=['age_median','age_mean','age_mode',"age_random",'age_end_dist',"age_arbitrary",'age_NaN']
for i in str[:-1]:
    mean=data[i].mean()
    std=data[i].std()
    upper=mean+3*std
    lower=mean-3*std
    data.loc[data[i]>upper,i]=upper
    data.loc[data[i]<lower,i]=lower


Thống kê độ chính xác của 14 tổ hợp làm sạch dữ liệu

In [118]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

DCX=[]
for i in str:
    arr1=[]
    arr2=[]
    if i=='age_NaN':
        X1=data_pre[['age_median','fare','age_NaN']]
        X2=data[['age_median','fare','age_NaN']]
    # get data
    X1=data_pre[[i,'fare']]
    X2=data[[i,'fare']]
    y=data['survived']
    # Thử nghiệm 10 lần
    for i in range(10):
        X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y,train_size=0.7, random_state=i)
        X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y,train_size=0.7, random_state=i)

        lr1=LogisticRegression(random_state=0)
        lr1.fit(X_train1,y_train1)
        lr1.predict(X_test1)
        arr1.append(lr1.score(X_test1,y_test1))

        lr2=LogisticRegression(random_state=0)
        lr2.fit(X_train2,y_train2)
        lr2.predict(X_test2)
        arr2.append(lr2.score(X_test2,y_test2))

    pre=round(np.mean(arr1),5)
    af=round(np.mean(arr2),5) 
    DCX.append([pre,af])

dic={}
for i in range(len(DCX)):
    dic[str[i]]=DCX[i]
table=pd.DataFrame(dic,index=['Chưa xử lý','Đã xử lý'])
table

,age_median,age_mean,age_mode,age_random,age_end_dist,age_arbitrary,age_NaN
Chưa xử lý,0.65336,0.65299,0.65336,0.65448,0.66381,0.66007,0.66791
Đã xử lý,0.66716,0.66716,0.66567,0.66791,0.66642,0.66381,0.67052


NHẬN XÉT
- Trường hợp DCX cao nhất: tạo đặc trưng mới có xử lý ngoại lệ
- Trường hợp DCX thấp nhất: Mean chưa xử lý ngoại lệ


# Phần 5.2


Đề xuất:
- Chuẩn hóa dữ liệu: Min-max
- Áp dụng logarit biến đổi fare về phân bố chuẩn
- Sử dụng trường hợp tạo đặc trưng mới có xử lý ngoại lệ

In [119]:
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import MinMaxScaler

min_max=MinMaxScaler()
df_pre=data_pre[['age_median','fare','age_NaN','survived']].copy()
df_pre['fare_log']=np.log1p(df_pre['fare'])

df_pre=pd.DataFrame(min_max.fit_transform(df_pre[['age_median','fare','fare_log','age_NaN','survived']]),
                                                columns=df_pre.columns)

X=df_pre[['age_median','fare_log','age_NaN']]
y=df_pre['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y,train_size=0.7, random_state=0)


lr=LogisticRegression(random_state=0)
result = cross_validate(lr, X, y,cv=10) # defaults to 5-fold CV
print("Accuracy mean: {}".format(round(np.mean(result['test_score']),3)))



Accuracy mean: 0.801


NHẬN XÉT:
- DCX cải thiện 13% 